In [9]:
# Step 1: Database Setup

import sqlite3

def setup_database():
    conn = sqlite3.connect('plant_disease_results.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS results (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        image_name TEXT,
                        prediction TEXT,
                        confidence REAL,
                        timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
                    )''')
    conn.commit()
    conn.close()

setup_database()

In [22]:
BATCH_SIZE=32
IMAGE_SIZE=256
CHANNELS = 3

In [3]:
import os
from tensorflow.keras.utils import image_dataset_from_directory

dataset_path = 'D:/PROJECTS/4th Year Project/kaggle/PlantVillage'

# Load and preprocess data
def load_data(dataset_path, img_size=(224, 224), batch_size=32, validation_split=0.2, seed=42):
    # Load train and validation datasets using validation_split
    train_ds = image_dataset_from_directory(
        dataset_path,
        image_size=img_size,
        batch_size=batch_size,
        validation_split=validation_split,
        subset="training",
        seed=seed
    )
    val_ds = image_dataset_from_directory(
        dataset_path,
        image_size=img_size,
        batch_size=batch_size,
        validation_split=validation_split,
        subset="validation",
        seed=seed
    )
    return train_ds, val_ds

# Specify dataset path
train_ds, val_ds = load_data(dataset_path)

# Display the structure of datasets
print(f"Training dataset: {train_ds}")
print(f"Validation dataset: {val_ds}")


Found 2152 files belonging to 3 classes.
Using 1722 files for training.
Found 2152 files belonging to 3 classes.
Using 430 files for validation.
Training dataset: <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
Validation dataset: <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [23]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])

NameError: name 'tf' is not defined

In [21]:
# Step 3: Data Augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create data augmentation generator
def augment_data(train_ds, img_size=(224, 224)):
    datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    augmented_data = datagen.flow_from_directory(
        train_ds,
        target_size=img_size,
        batch_size=32,
        class_mode='categorical'
    )
    return augmented_data

augmented_train_ds = augment_data(dataset_path)

Found 2152 images belonging to 3 classes.


In [20]:
# Step 4: Model Design
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build the CNN model
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)

NameError: name 'models' is not defined

In [5]:
from tensorflow.keras.optimizers import Adam

# Compile and train the model
def train_model(model, train_ds, val_ds, epochs=50):
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',  # Use sparse loss
                  metrics=['accuracy'])
    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)
    model.save('plant_disease_model.h5')
    return history

history = train_model(model, train_ds, val_ds)


Epoch 1/50


54/54 ━━━━━━━━━━━━━━━━━━━━ 50s 907ms/step - accuracy: 0.4801 - loss: 826.4453 - val_accuracy: 0.6814 - val_loss: 0.9789
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 51s 951ms/step - accuracy: 0.7367 - loss: 0.7033 - val_accuracy: 0.7116 - val_loss: 0.8542
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 49s 906ms/step - accuracy: 0.7899 - loss: 0.6054 - val_accuracy: 0.7302 - val_loss: 0.7368
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 48s 880ms/step - accuracy: 0.8404 - loss: 0.4731 - val_accuracy: 0.7349 - val_loss: 0.7294
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 47s 874ms/step - accuracy: 0.8716 - loss: 0.4003 - val_accuracy: 0.7395 - val_loss: 0.7108
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 49s 902ms/step - accuracy: 0.8950 - loss: 0.3198 - val_accuracy: 0.7233 - val_loss: 0.7563
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 47s 872ms/step - accuracy: 0.9205 - loss: 0.2962 - val_accuracy: 0.7349 - val_loss: 0.8797
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 48s 891ms/step - accuracy: 0.9092 - loss: 0.2860 - val_accuracy: 0.7

In [6]:
from tensorflow.keras.models import load_model
import numpy as np
from sklearn.metrics import classification_report

# Load the saved model
def evaluate_model(model_path, val_ds):
    model = load_model(model_path)
    predictions = []
    true_labels = []

    for images, labels in val_ds:
        preds = model.predict(images)
        predictions.extend(np.argmax(preds, axis=1))
        true_labels.extend(labels.numpy())  # No need for np.argmax here

    print(classification_report(true_labels, predictions))

evaluate_model('plant_disease_model.h5', val_ds)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
              precision    recall  f1-score   support

           0       0.78      0.88      0.83       182
           1       0.79      0.75      0.77       208
           2       0.44      0.30      0.36        40

    accuracy                           0.77       430
   macro avg       0.67      0.64      0.65       430
weighted avg       0.76      0.77      0.76       430



In [14]:
# Step 7: Testing with Individual Images
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(model_path, img_path, class_names):
    model = load_model(model_path)
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)
    confidence = predictions[0][predicted_class]

    print(f"Image: {img_path}")
    print(f"Predicted Class: {class_names[predicted_class]}")
    print(f"Confidence: {confidence:.2f}")

    # Save results to the database
    conn = sqlite3.connect('plant_disease_results.db')
    cursor = conn.cursor()
    cursor.execute('''INSERT INTO results (image_name, prediction, confidence) VALUES (?, ?, ?)''',
                   (os.path.basename(img_path), class_names[predicted_class], float(confidence)))
    conn.commit()
    conn.close()

# Example usage
class_names = ['Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight']  # Replace with your actual class names
predict_image('plant_disease_model.h5', 'D:/PROJECTS/4th Year Project/kaggle/PlantVillage/Potato___Early_blight/0c5b14d9-8b1c-4c39-bb23-1835b5760caa___RS_Early.B 7937.JPG', class_names)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Image: D:/PROJECTS/4th Year Project/kaggle/PlantVillage/Potato___Early_blight/0c5b14d9-8b1c-4c39-bb23-1835b5760caa___RS_Early.B 7937.JPG
Predicted Class: Potato___healthy
Confidence: 0.76
